**سلام علیکم**

In [1]:
!rm hetionet-v1.0-nodes.tsv  # اگه فایل قدیمی هست، پاکش کن

In [2]:
!wget https://github.com/hetio/hetionet/raw/master/hetnet/tsv/hetionet-v1.0-nodes.tsv


--2025-07-19 16:24:59--  https://github.com/hetio/hetionet/raw/master/hetnet/tsv/hetionet-v1.0-nodes.tsv
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hetio/hetionet/master/hetnet/tsv/hetionet-v1.0-nodes.tsv [following]
--2025-07-19 16:24:59--  https://raw.githubusercontent.com/hetio/hetionet/master/hetnet/tsv/hetionet-v1.0-nodes.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2509984 (2.4M) [text/plain]
Saving to: ‘hetionet-v1.0-nodes.tsv’

hetionet-v1.0-nodes 100%[===================>]   2.39M  --.-KB/s    in 0.05s   

2025-07-19 16:24:59 (51.4 MB/s) - ‘hetionet-v1.0-nodes.tsv’ saved [25

In [3]:
!wget https://github.com/hetio/hetionet/raw/master/hetnet/tsv/hetionet-v1.0-edges.sif.gz


--2025-07-19 16:24:59--  https://github.com/hetio/hetionet/raw/master/hetnet/tsv/hetionet-v1.0-edges.sif.gz
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/hetio/hetionet/master/hetnet/tsv/hetionet-v1.0-edges.sif.gz [following]
--2025-07-19 16:25:00--  https://media.githubusercontent.com/media/hetio/hetionet/master/hetnet/tsv/hetionet-v1.0-edges.sif.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12401407 (12M) [application/octet-stream]
Saving to: ‘hetionet-v1.0-edges.sif.gz’

hetionet-v1.0-edges 100%[===================>]  11.83M  --.-KB/s    in 0.08s   

2025-07-19 16:25:00

In [ ]:
!gunzip hetionet-v1.0-edges.sif.gz


gzip: hetionet-v1.0-edges.sif already exists; do you wish to overwrite (y or n)? 

In [ ]:
import pandas as pd
nodes = pd.read_csv('hetionet-v1.0-nodes.tsv', sep='\t', encoding='utf-8-sig')

print(nodes.columns)


In [ ]:
edges = pd.read_csv('hetionet-v1.0-edges.sif', sep='\t')
print(edges.columns)
print(edges.head())


In [ ]:
print("📦 Nodes:")
print(nodes.head())

print("🔗 Edges:")
print(edges.head())


In [ ]:
import networkx as nx

# ساخت گراف
G = nx.Graph()

# افزودن نودها
for _, row in nodes.iterrows():
    G.add_node(row['id'], name=row['name'], kind=row['kind'])

# افزودن یال‌ها
for _, row in edges.iterrows():
    G.add_edge(row['source'], row['target'], metaedge=row['metaedge'])


 pkl ذخیره گراف

In [ ]:
import pickle

with open("hetionet_graph.pkl", "wb") as f:
    pickle.dump(G, f)


لود گراف pkl

In [ ]:
import pickle

with open("hetionet_graph.pkl", "rb") as f:
    G = pickle.load(f)

# بازیابی با روش های مختلف

DFS

In [ ]:
from networkx.algorithms.traversal.depth_first_search import dfs_edges

def dfs_search(graph, start_node, max_depth=2):
    visited = set()
    result = []

    def dfs(node, depth):
        if depth > max_depth:
            return
        visited.add(node)
        result.append((node, depth))
        for neighbor in graph.neighbors(node):
            if neighbor not in visited:
                dfs(neighbor, depth + 1)

    dfs(start_node, 0)
    return result

# مثال:
start_node = 'Gene::3149'
dfs_result = dfs_search(G, start_node, max_depth=2)
for node, depth in dfs_result:
    print(f"Depth {depth}: {G.nodes[node]['kind']} - {G.nodes[node]['name']}")


مشکل:

نود های برگردونده شده خیلی زیادن

BFS

In [ ]:
from collections import deque

def bfs_search(graph, start_node, max_depth=2):
    visited = set()
    queue = deque([(start_node, 0)])
    result = []

    while queue:
        node, depth = queue.popleft()
        if node in visited or depth > max_depth:
            continue
        visited.add(node)
        result.append((node, depth))
        for neighbor in graph.neighbors(node):
            if neighbor not in visited:
                queue.append((neighbor, depth + 1))

    return result

# مثال:
start_node = 'Gene::3149'
bfs_result = bfs_search(G, start_node, max_depth=2)
for node, depth in bfs_result:
    print(f"Depth {depth}: {G.nodes[node]['kind']} - {G.nodes[node]['name']}")


بازیابی مسیرهای بین دو گره (کوتاه‌ترین مسیرها)

In [ ]:
def get_shortest_paths(graph, source, target, max_paths=3):
    try:
        paths = list(nx.all_shortest_paths(graph, source=source, target=target))
        return paths[:max_paths]
    except nx.NetworkXNoPath:
        return []

# مثال:
source = 'Gene::3149'
target = 'Biological Process::GO:0008150'
paths = get_shortest_paths(G, source, target)

for i, path in enumerate(paths):
    print(f"\nPath {i+1}:")
    for node in path:
        print(f"- {G.nodes[node]['kind']}: {G.nodes[node]['name']}")


گرفتن همسایه‌های مستقیم با نوع خاص

In [ ]:
def get_neighbors_by_type(graph, node_id, kind_filter=None):
    neighbors = []
    for neighbor in graph.neighbors(node_id):
        kind = graph.nodes[neighbor].get('kind')
        if kind_filter is None or kind == kind_filter:
            neighbors.append((neighbor, graph.nodes[neighbor]['name']))
    return neighbors

# مثال:
neighbors = get_neighbors_by_type(G, 'Gene::3149', kind_filter='Biological Process')
for nid, name in neighbors:
    print(f"{nid} - {name}")


تبدیل نتایج گراف به جملات متنی (برای RAG)

In [ ]:
def get_sentences_from_edges(graph, node_id, max_depth=1):
    bfs_result = bfs_search(graph, node_id, max_depth)
    sentences = []
    for node, depth in bfs_result:
        for neighbor in graph.neighbors(node):
            if node == neighbor:
                continue
            u = graph.nodes[node]
            v = graph.nodes[neighbor]
            edge_data = graph.get_edge_data(node, neighbor)
            sentences.append(f"{u['name']} --[{edge_data['metaedge']}]--> {v['name']}")
    return list(set(sentences))  # حذف تکراری‌ها

# مثال:
sentences = get_sentences_from_edges(G, 'Gene::3149', max_depth=2)
for s in sentences[:10]:
    print(s)


# استخراج توکن و جست و جو روابط

استخراج کلمات کلیدی از سؤال با spaCy

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def extract_keywords(text):
    doc = nlp(text)
    keywords = set()

    # 1. استخراج Named Entities
    for ent in doc.ents:
        if ent.label_ not in {"DATE", "TIME", "PERCENT", "MONEY", "QUANTITY", "ORDINAL", "CARDINAL"}:
            keywords.add(ent.text.lower())

    # 2. استخراج اسم‌ها و اسم خاص‌ها با POS
    for token in doc:
        if (
            token.pos_ in {"NOUN", "PROPN"} and
            token.text.lower() not in STOP_WORDS and
            token.is_alpha and
            len(token.text) > 2
        ):
            keywords.add(token.text.lower())

    # 3. استخراج کلمات از dependency head در سوالات
    for token in doc:
        if token.dep_ in {"nsubj", "dobj", "pobj", "attr", "ROOT"} and token.is_alpha:
            if token.text.lower() not in STOP_WORDS:
                keywords.add(token.text.lower())

    return sorted(keywords)
question = "Does metformin treat type 2 diabetes?"
tokens = extract_keywords(question)
print("🔍 Extracted tokens:", tokens)


بررسی وجود توکن‌ها در گراف

In [ ]:
def match_tokens_to_nodes(graph, tokens):
    matched = {}
    for token in tokens:
        token_lower = token.lower()
        for node_id, attrs in graph.nodes(data=True):
            if token_lower in attrs['name'].lower():
                matched[token] = node_id
                break
    return matched

# مثال:
matches = match_tokens_to_nodes(G, tokens)
print("✅ Matched nodes in graph:")
for token, node_id in matches.items():
    print(f"{token} → {node_id}")


 بررسی وجود ارتباط بین دو موجودیت

In [ ]:
def get_direct_or_shortest_relation(graph, node1, node2, max_depth=2):
    if graph.has_edge(node1, node2):
        return [(node1, node2)]
    try:
        path = nx.shortest_path(graph, source=node1, target=node2)
        return list(zip(path[:-1], path[1:]))
    except nx.NetworkXNoPath:
        return []

# اگر دو تا node داریم:
if len(matches) >= 2:
    token1, token2 = list(matches.values())[:2]
    relations = get_direct_or_shortest_relation(G, token1, token2)

    print(f"\n Path between {token1} and {token2}:")
    for u, v in relations:
        rel = G.get_edge_data(u, v)['metaedge']
        print(f"{G.nodes[u]['name']} --[{rel}]--> {G.nodes[v]['name']}")


مثال عملی

In [ ]:
question = "What is the relationship between HMGB3 and pulmonary valve formation?"

tokens = extract_keywords(question)
matches = match_tokens_to_nodes(G, tokens)

if len(matches) >= 2:
    node_ids = list(matches.values())
    relations = get_direct_or_shortest_relation(G, node_ids[0], node_ids[1])
    if relations:
        print(f"\n Path found between {tokens[0]} and {tokens[1]}:\n")
        for u, v in relations:
            rel = G.get_edge_data(u, v)['metaedge']
            print(f"{G.nodes[u]['name']} --[{rel}]--> {G.nodes[v]['name']}")
    else:
        print("No path found between matched nodes.")
else:
    print("Less than 2 tokens matched in the graph.")


1. استخراج هوشمند کلمات کلیدی از جمله (عبارات چندکلمه‌ای)

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def extract_keywords(text):
    doc = nlp(text)
    keywords = set()

    # عبارات اسمی کامل مثل "pulmonary valve formation"
    for chunk in doc.noun_chunks:
        phrase = chunk.text.lower().strip()
        if len(phrase.split()) <= 5 and not all(w in STOP_WORDS for w in phrase.split()):
            keywords.add(phrase)

    # موجودیت‌های نام‌دار (NER)
    for ent in doc.ents:
        keywords.add(ent.text.lower())

    # اسم‌ها و افعال کلیدی
    for token in doc:
        if token.pos_ in {"NOUN", "PROPN", "VERB"} and token.text.lower() not in STOP_WORDS:
            keywords.add(token.text.lower())

    return sorted(keywords)


2. تطبیق هوشمند با نودهای گراف (match_tokens_to_nodes)

In [ ]:
from difflib import get_close_matches

def match_tokens_to_nodes(graph, tokens):
    matches = {}
    all_node_names = {n: graph.nodes[n]['name'].lower() for n in graph.nodes}

    for token in tokens:
        # جستجوی تطبیقی (fuzzy match) با node name
        close = get_close_matches(token, all_node_names.values(), n=1, cutoff=0.8)
        if close:
            matched_name = close[0]
            for node_id, name in all_node_names.items():
                if name == matched_name:
                    matches[token] = node_id
                    break
    return matches


3. پیدا کردن مسیر بین دو نود در گراف

In [ ]:
import networkx as nx

def get_direct_or_shortest_relation(graph, source, target):
    if graph.has_edge(source, target):
        return [(source, target)]
    try:
        path = nx.shortest_path(graph, source, target)
        return list(zip(path, path[1:]))
    except nx.NetworkXNoPath:
        return []


In [ ]:
question = "What is the relationship between HMGB3 and pulmonary valve formation?"

tokens = extract_keywords(question)
print("Extracted Keywords:", tokens)

matches = match_tokens_to_nodes(G, tokens)
print("Matched Tokens:", {k: G.nodes[v]['name'] for k, v in matches.items()})

if len(matches) >= 2:
    node_ids = list(matches.values())
    relations = get_direct_or_shortest_relation(G, node_ids[0], node_ids[1])
    if relations:
        print(f"\nPath found between '{G.nodes[node_ids[0]]['name']}' and '{G.nodes[node_ids[1]]['name']}':\n")
        for u, v in relations:
            rel = G.get_edge_data(u, v).get('metaedge', 'related_to')
            print(f"{G.nodes[u]['name']} --[{rel}]--> {G.nodes[v]['name']}")
    else:
        print("No path found between matched nodes.")
else:
    print("Less than 2 tokens matched in the graph.")


مشکل:
کلمه "formation" به اشتباه به Formication (احساس خزیدن روی پوست!) مچ شده، چون از نظر نوشتاری شبیه هست. ولی این ربطی به بیولوژی قلب یا دریچه نداره.
مسیر تولیدشده:

Formication → Aripiprazole → PCNA → HMGB3
 مشکل شدید:
این مسیر از یک احساس پوستی به یک دارو روان‌پزشکی تا برسه به ژن — یعنی کاملاً بی‌ربط به موضوع اصلی:
"رابطه HMGB3 و تشکیل دریچه ریوی".

 راه‌حل 1: فقط عبارات علمی چندکلمه‌ای مهم را نگه داریم

In [ ]:
def extract_important_phrases(text):
    doc = nlp(text)
    keywords = set()

    for chunk in doc.noun_chunks:
        phrase = chunk.text.lower().strip()
        if len(phrase.split()) > 1 and not any(w in STOP_WORDS for w in phrase.split()):
            keywords.add(phrase)

    # همچنین اجازه بدهیم برخی توکن‌های خیلی مهم تکی هم وارد شوند
    for token in doc:
        if token.text.lower() in {'hmgb3'}:
            keywords.add(token.text.lower())

    return sorted(keywords)


 راه‌حل 2: هنگام تطبیق، فقط به توکن‌هایی که دقیقاً match دارند اولویت بده

In [ ]:
def match_tokens_to_nodes_strict(graph, tokens):
    matches = {}
    node_names = {n: graph.nodes[n]['name'].lower() for n in graph.nodes}

    for token in tokens:
        exact_matches = [n for n, name in node_names.items() if token == name]
        if exact_matches:
            matches[token] = exact_matches[0]  # اولین تطبیق
    return matches


In [ ]:
question = "What is the relationship between HMGB3 and pulmonary valve formation?"

tokens = extract_important_phrases(question)
print("Filtered Keywords:", tokens)

matches = match_tokens_to_nodes_strict(G, tokens)
print("Matched Tokens:", {k: G.nodes[v]['name'] for k, v in matches.items()})

if len(matches) >= 2:
    node_ids = list(matches.values())
    relations = get_direct_or_shortest_relation(G, node_ids[0], node_ids[1])
    if relations:
        print(f"\n Path found between '{G.nodes[node_ids[0]]['name']}' and '{G.nodes[node_ids[1]]['name']}':\n")
        for u, v in relations:
            rel = G.get_edge_data(u, v).get('metaedge', 'related_to')
            print(f"{G.nodes[u]['name']} --[{rel}]--> {G.nodes[v]['name']}")
    else:
        print("No path found between matched nodes.")
else:
    print("Less than 2 tokens matched in the graph.")


هوشمندتر با استفاده از مدل‌های BioNLP یا BioBERT انجام بشه

نصب کتابخانه‌ها

In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz


In [ ]:
import spacy

# مدل NER تخصصی برای علوم زیستی
nlp = spacy.load("en_ner_bionlp13cg_md")

def extract_biomedical_entities(text):
    doc = nlp(text)
    keywords = set()

    # استخراج موجودیت‌های زیستی
    for ent in doc.ents:
        if ent.label_ in {
            "GENE_OR_GENE_PRODUCT",
            "CELLULAR_COMPONENT",
            "SIMPLE_CHEMICAL",
            "DISEASE_OR_DISORDER",
            "ANATOMICAL_SYSTEM",
            "BIOLOGICAL_PROCESS",
            "CELL",
            "TISSUE"
        }:
            keywords.add(ent.text.lower())

    return sorted(keywords)


In [ ]:
question = "What is the relationship between HMGB3 and pulmonary valve formation?"
tokens = extract_biomedical_entities(question)

print("🧬 Extracted Biomedical Tokens:", tokens)


 افزودن semantic similarity با BioBERT یا PubMedBERT

In [ ]:
!pip install transformers
!pip install torch


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# بارگذاری مدل BioBERT یا PubMedBERT
model_name = "dmis-lab/biobert-base-cased-v1.1"  # یا: "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


In [ ]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=32)
    with torch.no_grad():
        outputs = model(**inputs)
    # استفاده از [CLS] توکن برای خلاصه‌سازی کل جمله
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().numpy()


In [ ]:
node_embeddings = {}
for node_id, data in G.nodes(data=True):
    name = data['name']
    try:
        node_embeddings[node_id] = get_bert_embedding(name)
    except:
        continue  # اگر خطایی در توکنایزر بود


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_closest_node(token, node_embeddings, topk=3):
    token_emb = get_bert_embedding(token).reshape(1, -1)
    sims = {
        node_id: cosine_similarity(token_emb, emb.reshape(1, -1))[0][0]
        for node_id, emb in node_embeddings.items()
    }
    top_matches = sorted(sims.items(), key=lambda x: x[1], reverse=True)[:topk]
    return top_matches


نمونه استفاده:

In [ ]:
token = "pulmonary valve formation"
top_matches = find_closest_node(token, node_embeddings)

print(f"\n🔍 Top matches for: '{token}'")
for node_id, score in top_matches:
    print(f"{G.nodes[node_id]['name']} (score: {score:.3f})")


وزن مسیر (مثلاً مجموع اعتماد به تعاملات)

نمایش بصری مسیر

مسیرهای جایگزین (top-3 shortest)

# کارهای واجب
1.استخراج بهتر کنیم

2.dfs و bfs خروجیشون خیلی زیاده

3.مدل جناروتور

4.گزارش کارهایی که تا الان کردیم

5.ایمیل به بیگی

6.الگوریتم های بازیابی از گراف(ترکیبی و وزن دار و دایجسترا و ...)

7.قبل از بازیابی نود یه دور بدیم به یه مدل gpt